In [9]:
# https://docs.microsoft.com/zh-tw/azure/machine-learning/how-to-use-parallel-run-step
from azureml.core import Datastore
from azureml.core import Workspace
ws = Workspace.from_config()

mnist_blob = Datastore.register_azure_blob_container(ws, 
                      datastore_name="mnistdatastore", 
                      container_name="mnistdata", 
                      account_name="lab04astorage",
                      account_key='q46wVHJlcwlfeab2yg2dqPePM4RmEq8J26Bp4fN86luRKr/ign0SSBcYbVBui0fHrQ5tkVxY+naBC/2OLdsBkA==',
                      overwrite=True
                      )
def_data_store = Datastore.get(ws, 'mnistdatastore')

In [21]:
from azureml.core.dataset import Dataset

path_on_datastore = mnist_blob.path('mnist/')
input_mnist_ds = Dataset.File.from_files(path=path_on_datastore, validate=False)

In [22]:
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter

pipeline_param = PipelineParameter(name="mnist_param", default_value=input_mnist_ds)
input_mnist_ds_consumption = DatasetConsumptionConfig("minist_param_config", pipeline_param).as_mount()

In [23]:
from azureml.pipeline.core import Pipeline, PipelineData

output_dir = PipelineData(name="inferences", datastore=def_data_store)

In [24]:
import os
import tarfile
import urllib.request

model_dir = 'models'
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

url="https://pipelinedata.blob.core.windows.net/mnist-model/mnist-tf.tar.gz"
response = urllib.request.urlretrieve(url, "model.tar.gz")
tar = tarfile.open("model.tar.gz", "r:gz")
tar.extractall(model_dir)

In [25]:
from azureml.core.model import Model

# Register the downloaded model 
model = Model.register(model_path="models/",
                       model_name="mnist",
                       tags={'pretrained': "mnist"},
                       description="Mnist trained tensorflow model",
                       workspace=ws)

Registering model mnist


In [26]:
%%writefile digit_identification.py

# Snippets from a sample script.
# Refer to the accompanying digit_identification.py
# (https://aka.ms/batch-inference-notebooks)
# for the implementation script.

import os
import numpy as np
import tensorflow as tf
from PIL import Image
from azureml.core import Model


def init():
    global g_tf_sess

    # Pull down the model from the workspace
    model_path = Model.get_model_path("mnist")

    # Construct a graph to execute
    tf.reset_default_graph()
    saver = tf.train.import_meta_graph(os.path.join(model_path, 'mnist-tf.model.meta'))
    g_tf_sess = tf.Session()
    saver.restore(g_tf_sess, os.path.join(model_path, 'mnist-tf.model'))


def run(mini_batch):
#     print(f'run method start: {__file__}, run({mini_batch})')
    print(f'run method start')
    resultList = []
    in_tensor = g_tf_sess.graph.get_tensor_by_name("network/X:0")
    output = g_tf_sess.graph.get_tensor_by_name("network/output/MatMul:0")

    for image in mini_batch:
        # Prepare each image
        data = Image.open(image)
        np_im = np.array(data).reshape((1, 784))
        # Perform inference
        inference_result = output.eval(feed_dict={in_tensor: np_im}, session=g_tf_sess)
        # Find the best probability, and add it to the result list
        best_result = np.argmax(inference_result)
        resultList.append("{}: {}".format(os.path.basename(image), best_result))

    return resultList

Overwriting digit_identification.py


In [27]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

batch_conda_deps = CondaDependencies.create(pip_packages=["tensorflow==1.15.2", "pillow", 
                                                          "azureml-core", "azureml-dataset-runtime[pandas, fuse]"])

batch_env = Environment(name="batch_environment")
batch_env.python.conda_dependencies = batch_conda_deps
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_CPU_IMAGE

In [20]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "LAB05B-Cluster" # change to your compute cluster name

# Verify that cluster exists
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=2)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

#if compute_name in ws.compute_targets:
#    compute_target = ws.compute_targets[compute_name]
#    if compute_target and type(compute_target) is AmlCompute:
#        print('found compute target. just use it. ' + compute_name)
#else:
#    print('creating a new compute target...')
#    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
#                                                                min_nodes = compute_min_nodes, 
#                                                                max_nodes = compute_max_nodes)
#
#    # create the cluster
#    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
#    
#    # can poll for a minimum number of nodes and for a specific timeout. 
#    # if no min node count is provided it will use the scale settings for the cluster
#    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
#    
#     # For a more detailed view of current AmlCompute status, use get_status()
#    print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-08-02T08:26:08.535000+00:00', 'errors': None, 'creationTime': '2020-08-02T06:28:59.179593+00:00', 'modifiedTime': '2020-08-02T06:29:16.479532+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 2, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS2_V2'}


In [28]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import ParallelRunConfig

parallel_run_config = ParallelRunConfig(
    source_directory='.',
    entry_script="digit_identification.py",
    mini_batch_size=PipelineParameter(name="batch_size_param", default_value="5"),
    error_threshold=10,
    output_action="append_row",
    append_row_file_name="mnist_outputs.txt",
    environment=batch_env,
    compute_target=compute_target,
    process_count_per_node=PipelineParameter(name="process_count_param", default_value=2),
    node_count=2)

In [29]:
from azureml.pipeline.steps import ParallelRunStep

parallelrun_step = ParallelRunStep(
    name="predict-digits-mnist",
    parallel_run_config=parallel_run_config,
    inputs=[input_mnist_ds_consumption],
    output=output_dir,
    allow_reuse=True
)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/pipeline/core/_parallel_run_step_base.py:555: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse] for file dataset.
Please add relevant package in CondaDependencies.
  Please add relevant package in CondaDependencies.""".format(extra, self._input_ds_type), UserWarning)


In [30]:
from azureml.pipeline.core import Pipeline
from azureml.core.experiment import Experiment

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
experiment = Experiment(ws, 'digit_identification')
pipeline_run = experiment.submit(pipeline)

WARNING - 'azureml-defaults' is not included in the pip package list in the environment definition in RunConfiguration. This may result in job failures during execution.


Created step predict-digits-mnist [b181fa79][697b107a-ba59-47b5-964d-1e32ba217f70], (This step will run and generate new outputs)
Submitted PipelineRun 3e9a0fd4-d35e-42c3-bc08-2788dc7ebab2
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/digit_identification/runs/3e9a0fd4-d35e-42c3-bc08-2788dc7ebab2?wsid=/subscriptions/35241d74-3b9e-4778-bb92-4bb15e7b0410/resourcegroups/DP-100/workspaces/Lab01A


In [31]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

pipeline_run.wait_for_completion(show_output=True)

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 3e9a0fd4-d35e-42c3-bc08-2788dc7ebab2
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/digit_identification/runs/3e9a0fd4-d35e-42c3-bc08-2788dc7ebab2?wsid=/subscriptions/35241d74-3b9e-4778-bb92-4bb15e7b0410/resourcegroups/DP-100/workspaces/Lab01A
PipelineRun Status: Running


StepRunId: 11e7ea3f-3758-4224-935a-614a78e5a267
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/digit_identification/runs/11e7ea3f-3758-4224-935a-614a78e5a267?wsid=/subscriptions/35241d74-3b9e-4778-bb92-4bb15e7b0410/resourcegroups/DP-100/workspaces/Lab01A
StepRun( predict-digits-mnist ) Status: NotStarted
StepRun( predict-digits-mnist ) Status: Queued

Streaming azureml-logs/20_image_build_log.txt
2020/08/02 09:11:14 Downloading source code...
2020/08/02 09:11:15 Finished downloading source code
StepRun( predict-digits-mnist ) Status: Running
2020/08/02 09:11:16 Creating Docker network: acb_default_network, driver: 'bridge'
2020/08/02 09:11:16 Successful


#
# To activate this environment, use
#
#     $ conda activate /azureml-envs/azureml_40740abd0db39f33e22f8442e072813e
#
# To deactivate an active environment, use
#
#     $ conda deactivate



==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda


Removing intermediate container f5fbd106f4c6
 ---> 10d36a8966ad
Step 9/15 : ENV PATH /azureml-envs/azureml_40740abd0db39f33e22f8442e072813e/bin:$PATH
 ---> Running in cb31623b7605
Removing intermediate container cb31623b7605
 ---> 6fe0cd66471a
Step 10/15 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_40740abd0db39f33e22f8442e072813e
 ---> Running in 0c5d2269c593
Removing intermediate container 0c5d2269c593
 ---> 05ee0f65f702
Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_40740abd0db39f33e22f8442e072813e/lib:$LD_LIBRARY_PATH
 ---> Running in 8a60cb2dfccd
Removing intermediate container 8a60cb2dfccd
 

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [ ]:
path_on_datastore = mnist_blob.path('mnist/0.png')
single_image_ds = Dataset.File.from_files(path=path_on_datastore, validate=False)

pipeline_run_2 = experiment.submit(pipeline, 
                                   pipeline_parameters={"mnist_param": single_image_ds, 
                                                        "batch_size_param": "1",
                                                        "process_count_param": 1}
                                  )

pipeline_run_2.wait_for_completion(show_output=True)

In [ ]:
import pandas as pd
import tempfile

batch_run = pipeline_run.find_step_run(parallelrun_step.name)[0]
batch_output = batch_run.get_output_data(output_dir.name)

target_dir = tempfile.mkdtemp()
batch_output.download(local_path=target_dir)
result_file = os.path.join(target_dir, batch_output.path_on_datastore, parallel_run_config.append_row_file_name)

df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["Filename", "Prediction"]
print("Prediction has ", df.shape[0], " rows")
df.head(10)